In [76]:
import numpy as np
import math
d1 = open('d1.txt','r')
d2 = open('d2.txt','r')
d3 = open('d3.txt','r')
d4 = open('d4.txt','r')
d5 = open('d5.txt','r')
dq = open('d_query.txt','r')
def get_words(fileid):
    words = []
    temp = []
    while(True):
        r = fileid.read(1)
        if(r.isalpha()):
            temp.append(r)
        else:
            word = ''.join(temp)
            if(word!=""):
                in_lowercase = word.casefold()
                words.append(in_lowercase)
                temp = []
        if(r==''):
            break
    return words
D1 = get_words(d1)
D2 = get_words(d2)
D3 = get_words(d3)
D4 = get_words(d4)
D5 = get_words(d5)
DQ = get_words(dq)


In [77]:
documents = [DQ,D1,D2,D3,D4,D5]



In [7]:
import nltk
nltk.download('punkt')
import string
import math
from nltk import FreqDist;
import numpy as np

input_files = ['d1.txt', 'd2.txt', 'd3.txt', 'd4.txt', 'd5.txt'];
all_words = [];

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\abood\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [8]:
# extract words from documents
for files in input_files:
  f = open(files, 'r');
  temp = f.read();
  f.close();
  temp1 = temp.translate(str.maketrans("","", string.punctuation));
  words = nltk.word_tokenize(temp1);
  all_words = all_words + words;
  words = [];

all_words = [i.lower() for i in all_words];
all_words.sort();

#print len(all_words)
dictofwords = list(set(all_words));
dictofwords.sort();
#print (dictofwords)

In [9]:
#convert documents to vectors
query = ['d_query.txt'];
all_files = input_files + query;
matrixofdocs = np.zeros((len(dictofwords), len(all_files)));

row = 0;
col = 0;

for i in all_files:
   f = open(i, 'r');
   temp = f.read();
   f.close();

   temp = temp.translate(str.maketrans("","",string.punctuation));

   words = nltk.word_tokenize(temp);
   words.sort();
   words = [w.lower() for w in words];

   dist = FreqDist(words)
   row = 0;

   for j in dictofwords:
    if j not in words:
      matrixofdocs[row][col] = 0;
    else:
      matrixofdocs[row][col] += dist[j];
    row += 1;

   col += 1

In [10]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
#from sklearn.feature_extraction.text import word2vec


def wordCount(document):
    
    wordvec = {}
    for i in document:
        if i in wordvec:
            wordvec[i] += 1
        else:
            wordvec[i] = 1
    
    # Computes tf for each word
   # for word in reviewTFDict:
      #  reviewTFDict[word] = reviewTFDict[word] / len(review)
  #  return reviewTFDict
    #for i in wordvec:
        #wordvec[i] = wordvec[i] / len(document)
    
    return wordvec
    

In [12]:
def computetf (document):
    
    wordvec = {}
    for i in document:
        if i in wordvec:
            wordvec[i] += 1
        else:
            wordvec[i] = 1
    
    
    for i in wordvec:
        wordvec[i] = wordvec[i] / len(document)
    
    
    return wordvec

In [135]:
D1tf= computetf(D1)
D2tf= computetf(D2)
D3tf= computetf(D3)
D4tf= computetf(D4)
D5tf= computetf(D5)
DQtf= computetf(DQ)
tfdocs = np.array([DQtf,D1tf,D2tf,D3tf,D4tf,D5tf])


In [15]:
def CountDict():
    countDict = {}
    #for i in dictofwords:
    for document in tfdocs:
            
        for word in document:
            if word in countDict:
                countDict[word] += 1
            else:
                countDict[word] = 1
    return countDict
        
countDict = CountDict()    

In [17]:
def computeIDFDict():
    """ Returns a dictionary whose keys are all the unique words in the
    dataset and whose values are their corresponding idf.
    """
    idfDict = {}
    for word in countDict:
       # print(countDict[word])
        if countDict[word] > 0:
            idfDict[word] = 1.0 + math.log(float(len(documents)) / countDict[word])
        else:
             idfDict[word] = 1.0
           
        
    return idfDict
  
  #Stores the idf dictionary
idfDict = computeIDFDict()

In [19]:
def computeReviewTFIDFDict(document):
    """ Returns a dictionary whose keys are all the unique words in the
    review and whose values are their corresponding tfidf.
    """
    reviewTFIDFDict = {}
    wordvec = computetf(document)
    #For each word in the review, we multiply its tf and its idf.
    for word in wordvec:
        reviewTFIDFDict[word] = wordvec[word] * idfDict[word]
    return reviewTFIDFDict

  #Stores the TF-IDF dictionaries
tfidfDict = [computeReviewTFIDFDict(wordvec) for wordvec in tfdocs]

In [96]:
wordDict = sorted(countDict.keys())
#print(wordDict)
def computeTFIDFVector(document):
    tfidfVector = [0.0] * len(wordDict)

      # For each unique word, if it is in the review, store its TF-IDF value.
    for i, word in enumerate(wordDict):
        if word in document:
            tfidfVector[i] = document[word]
    return tfidfVector

tfidfVector = [computeTFIDFVector(document) for document in tfidfDict]

In [98]:
def cosine_similarity(vector1, vector2):
    dot_product = sum(p*q for p,q in zip(vector1, vector2))
    magnitude = math.sqrt(sum([val**2 for val in vector1])) * math.sqrt(sum([val**2 for val in vector2]))
    if not magnitude:
        return 0
    return dot_product/magnitude

In [133]:
cos = np.zeros((6,6))
#print(cos)
#benchot = [][]
row = 0;
col = 0;
for i in range (len(tfdocs)):
    
    doc = tfdocs[i]
    TFIDF1 = computeTFIDFVector(doc)
    
    for j in range (len(tfdocs)):
        doc2 = tfdocs[j]
        TFIDF2 = computeTFIDFVector(doc2)
        cos[i][j] += round(cosine_similarity(TFIDF1, TFIDF2),3)
        j+=1
i+=1        
       
    #arr_flat = np.append(cos,sim)
#print(benchot)

In [134]:
cos

array([[1.   , 0.152, 0.119, 0.116, 0.312, 0.259],
       [0.152, 1.   , 0.767, 0.667, 0.568, 0.641],
       [0.119, 0.767, 1.   , 0.72 , 0.561, 0.567],
       [0.116, 0.667, 0.72 , 1.   , 0.643, 0.568],
       [0.312, 0.568, 0.561, 0.643, 1.   , 0.527],
       [0.259, 0.641, 0.567, 0.568, 0.527, 1.   ]])